In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cloudvolume import CloudVolume as cv
from caveclient import CAVEclient
from scipy.spatial import KDTree
import pygsheets
import ast
from collections import Counter, defaultdict
import time
import pickle
from scipy.spatial.distance import cdist
import networkx as nx
import random
from itertools import combinations
import scipy.sparse as sp

vol_graphene =cv('graphene://https://minnie.microns-daf.com/segmentation/table/zheng_ca3', agglomerate=True, use_https=True)
vol = cv('gs://zheng_mouse_hippocampus_production/v2/seg_m195',parallel=True, progress=False, use_https=True)
client = CAVEclient('zheng_ca3')


Process ForkPoolWorker-3:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-9:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-10:
Process ForkPoolWorker-12:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Process ForkPoolWorker-16:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anac

  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/opt/anaconda3/envs/allensdk/lib/python3.10/site-packages/multiprocess/queues.py",

In [2]:
# Load previously generated adjmatrix, mf_ids, pc_ids, dist_mat
syn_thresh = 3
with open('./variables/adjmat_and_bouton_thorn_dist_matrix_250619_th' +str(syn_thresh)+'.pkl','rb') as f:
    adj_mat, mf_ids, pc_ids, coord_df, d_mat = pickle.load(f)

In [41]:
# For each bouton, find number of pyr cells within distance r

def find_pyr_cells_near_MF_bouton(d_mat, r, post_ids):
    # d_mat: rows are boutons, columns are thorns
    # r: search distance
    # post_ids: coord_df['post_ids'] representing the pyr cell IDs of thorns

    nearby_pyr_cells = {}
    
    for bouton_idx, row in enumerate(d_mat):
        thorn_indices = np.where(row <= r)[0]
        pyr_cell_ids = ([post_ids[i] for i in thorn_indices])
        nearby_pyr_cells[bouton_idx] = pyr_cell_ids
        
    return nearby_pyr_cells

r = list(range(10,101,10))
#r = [5]
post_ids = coord_df['post_ids'].values
#print(f"Number of pyr cells within R micrometers of MF boutons")
print(f"Probability of bouton matched to its original pyr cells within R micromenters")
for i in range(len(r)):
    nearby_pyr_cells_of_boutons = find_pyr_cells_near_MF_bouton(d_mat, r[i], post_ids)
    n_nearby_pyr_cells = []
    prob_targeting_original_pyr = []
    for key, val in nearby_pyr_cells_of_boutons.items():
        if hasattr(val, '__len__') and not isinstance(val, (int, float, bool)):
            val_unique = list(set(val))
            n_nearby_pyr_cells.append(len(val_unique))
            n_thorns_back_to_itself = nearby_pyr_cells_of_boutons[key].count(post_ids[key])
            n_thorns_all = len(nearby_pyr_cells_of_boutons[key])
            prob_targeting_original_pyr.append(round(n_thorns_back_to_itself/n_thorns_all,3))
    #print(f"""{r[i]} um:  Mean = {np.mean(n_nearby_pyr_cells):.2f}, std dev = {np.std(n_nearby_pyr_cells):.2f}""")
    
    print(f"""{r[i]} um: mean prob = {np.mean(prob_targeting_original_pyr):.3f}, std dev = {np.std(prob_targeting_original_pyr):.3f}""")

Probability of bouton matched to its original pyr cells within R micromenters
10 um: mean prob = 0.433, std dev = 0.243
20 um: mean prob = 0.189, std dev = 0.150
30 um: mean prob = 0.105, std dev = 0.087
40 um: mean prob = 0.066, std dev = 0.051
50 um: mean prob = 0.046, std dev = 0.033
60 um: mean prob = 0.034, std dev = 0.025
70 um: mean prob = 0.026, std dev = 0.020
80 um: mean prob = 0.022, std dev = 0.017
90 um: mean prob = 0.018, std dev = 0.015
100 um: mean prob = 0.016, std dev = 0.013


In [32]:
Number of pyr cells within R micrometers of MF boutons
10 um:  Mean = 5.16, std dev = 2.46
20 um:  Mean = 13.99, std dev = 5.24
30 um:  Mean = 24.65, std dev = 7.71
40 um:  Mean = 36.54, std dev = 10.08
50 um:  Mean = 49.18, std dev = 12.73
60 um:  Mean = 62.30, std dev = 15.90
70 um:  Mean = 75.53, std dev = 19.66
80 um:  Mean = 88.80, std dev = 24.05
90 um:  Mean = 102.08, std dev = 28.93
100 um:  Mean = 115.40, std dev = 34.16

0.45

In [17]:
nearby_pyr_cells_of_boutons

{0: [648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346440668261,
  648518346440668261,
  648518346434626934],
 1: [648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690],
 2: [648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346440668261,
  648518346440668261,
  648518346440668261,
  648518346440668261,
  648518346434626934],
 3: [648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690],
 4: [648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690],
 5: [648518346439062690,
  648518346439062690,
  648518346439062690,
  648518346439062690,
  6485183

In [ ]:
# N trials of physically-constrained randomized connectivity
# Shuffle the connections between bouton and thorns within a radius
# Aim for as many one-to-one matches as possible
# Try to minimize the mapping of many boutons onto one thorn

N = 100
r = 10  # um
d_mat_thresh = (d_mat < r).astype(int)
bouton_indices, thorn_indices = np.where(d_mat_thresh == 1)
edges = [(f"b{i}", f"t{j}") for i, j in zip(bouton_indices, thorn_indices)]
counters = []

for ii in range(0,N):
    if ii % 10 == 0:
        print('trial number: ',ii)
    random.seed(ii)
    random.shuffle(edges)
    bouton_matched = set()
    thorn_matched = set()
    bouton_to_thorn = {f"b{i}": [] for i in range(d_mat_thresh.shape[0])}

    for b, t in edges:
        if b not in bouton_matched and t not in thorn_matched:
            bouton_to_thorn[b].append(t)
            bouton_matched.add(b)
            thorn_matched.add(t)

    for b, t in edges:
        if b not in bouton_matched:
            bouton_to_thorn[b].append(t)
            bouton_matched.add(b)

    thorn_to_bouton = defaultdict(list)
    for k, vals in bouton_to_thorn.items():
        for v in vals:
            thorn_to_bouton[v].append(k)

    # create adj matrix from shuffled bouton-thorn mapping
    target_mf_ids = coord_df['pre_ids'].unique()
    target_pyr_ids = coord_df['post_ids'].unique()
    adj_mat_rand = np.zeros((len(target_pyr_ids), len(target_mf_ids)), dtype=int)

    mf_to_pyr_randomized = {i: [] for i in range(len(target_mf_ids))}
    for key, val in bouton_to_thorn.items():
        key = int(key.lstrip('b'))
        val = int(val[0].lstrip('t'))
        this_mf_id = coord_df['pre_ids'].values[key]
        this_pyr_id = coord_df['post_ids'].values[val]
        idx_mf = np.where(target_mf_ids==this_mf_id)[0][0]
        idx_pyr = np.where(target_pyr_ids==this_pyr_id)[0][0]
        mf_to_pyr_randomized[idx_mf].append(idx_pyr)

    for key, vals in mf_to_pyr_randomized.items():
        for val in vals:
            adj_mat_rand[val,key] = 1
    print('Randomizing adjacency matrix with structural constraint, done!')

    # Find all pairs of pyr cells that share at least two common MFs
    pyr_pairs_rand = []
    shared_mfs_rand = []

    for idx_pyr1, idx_pyr2 in combinations(range(len(target_pyr_ids)),2):
        common_mf_indices = np.where((adj_mat_rand[idx_pyr1,:] == 1) & (adj_mat_rand[idx_pyr2,:] == 1))[0]
        common_mfs = []
        if len(common_mf_indices) >= 2:
            for i in range(len(common_mf_indices)):
                common_mfs.append(target_mf_ids[common_mf_indices[i]])
            pyr_pairs_rand.append([target_pyr_ids[idx_pyr1], target_pyr_ids[idx_pyr2]])
            shared_mfs_rand.append(common_mfs)

    num_shared_MFs = [len(sublist) for sublist in shared_mfs_rand]
    freq_num_shared_MFs = Counter(num_shared_MFs)
    counters.append(freq_num_shared_MFs)

In [ ]:
# Collect all unique keys
all_keys = set().union(*[counter.keys() for counter in counters])

# Calculate mean and standard deviation
stats = {}
for key in all_keys:
    # Get the values for the key from all counters (default to 0 if key is missing)
    values = [counter.get(key, 0) for counter in counters]
    
    # Compute mean and standard deviation
    mean = np.mean(values)
    std_dev = np.std(values, ddof=1)  # ddof=1 for sample standard deviation
    stats[key] = {"mean": mean, "std_dev": std_dev}

# Display results
print("Stats from " + str(N) + " trials of randomized connectivity with physical constraint of radius " + str(r) + "um")
for key, stat in stats.items():
    print(f"Number of shared MFs: {key}, Mean number of pyr pairs: {stat['mean']:.2f}, Std Dev: {stat['std_dev']:.2f}")

In [ ]:
# small test case

matrix = np.random.randint(0, 6, size=(10, 10))
#print(matrix)
d_mat_thresh = (matrix < 3).astype(int)
print(d_mat_thresh)

bouton_indices, thorn_indices = np.where(d_mat_thresh == 1)
edges = [(f"b{i}", f"t{j}") for i, j in zip(bouton_indices, thorn_indices)]
random.shuffle(edges)

bouton_matched = set()
thorn_matched = set()
bouton_to_thorn = {f"b{i}": [] for i in range(d_mat_thresh.shape[0])}
print('matching...')


stime = time.time() 
for b, t in edges:
    if b not in bouton_matched and t not in thorn_matched:
        bouton_to_thorn[b].append(t)
        bouton_matched.add(b)
        thorn_matched.add(t)
        
for b, t in edges:
    if b not in bouton_matched:
        bouton_to_thorn[b].append(t)
        bouton_matched.add(b)
        
print(bouton_to_thorn)

In [ ]:
lengths = [len(v) for v in bouton_to_thorn.values()]
length_counts = Counter(lengths)
print(length_counts)